In [ ]:
from dowhy import CausalModel
from dowhy.causal_identifier import (
    identify_effect,
    BackdoorIdentifier,
    BackdoorAdjustmentMethod,
    IDIdentifier,
    CausalIdentifierEstimandType,
)

In [ ]:
from dowhy.causal_graph import CausalGraph
import dowhy.datasets

data = dowhy.datasets.linear_dataset(
    beta=4,
    num_common_causes=2,
    num_instruments=1,
    num_effect_modifiers=1,
    num_samples=500,
    treatment_is_binary=True,
    stddev_treatment_noise=10,
    num_discrete_common_causes=1,
)

In [ ]:
treatment_name = data["treatment_name"]
outcome_name = data["outcome_name"]

graph = CausalGraph(
    treatment_name=treatment_name,
    outcome_name=outcome_name,
    graph=data["gml_graph"],
    effect_modifier_names=data["effect_modifier_names"],
    common_cause_names=data["common_causes_names"],
    observed_node_names=data["df"].columns.tolist(),
)

In [ ]:
# New functional API

identified_estimand = identify_effect(
    graph=graph,
    treatment=treatment_name,
    outcome=outcome_name,
    method=BackdoorIdentifier(estimand_type=CausalIdentifierEstimandType.NONPARAMETRIC_ATE),
)
print(identified_estimand)

In [ ]:
# Backwards compatibility check

model = CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"],
)

identified_estimand = model.identify_effect()
print(identified_estimand)

In [ ]:
# Another way of executing the identify effect by directly calling the object

identifier = BackdoorIdentifier(
    estimand_type=CausalIdentifierEstimandType.NONPARAMETRIC_ATE, backdoor_adjustment=BackdoorAdjustmentMethod.BACKDOOR_DEFAULT
)

identified_estimand = identifier.identify_effect(
    graph=graph,
    treatment_name=treatment_name,
    outcome_name=outcome_name,
)

print(identified_estimand)

In [ ]:
# New functional API (IDIdentifier)
identified_estimand = identify_effect(
    graph=graph,
    treatment=treatment_name,
    outcome=outcome_name,
    method=IDIdentifier(estimand_type=CausalIdentifierEstimandType.NONPARAMETRIC_ATE),
)
print(identified_estimand)

In [ ]:
# Backwards compatibility check (IDIdentifier)

model = CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"],
)

identified_estimand = model.identify_effect(method_name="id-algorithm")
print(identified_estimand)

In [ ]:
# Another way of executing the identify effect by directly calling the object

identifier = IDIdentifier(estimand_type=CausalIdentifierEstimandType.NONPARAMETRIC_ATE)

identified_estimand = identifier.identify_effect(
    graph=graph,
    treatment_name=treatment_name,
    outcome_name=outcome_name,
)

print(identified_estimand)